<a href="https://colab.research.google.com/github/YujiaLIAO-1/housing/blob/main/assignment03_task2_starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import packages
import pandas as pd

In [2]:
# read in the train dataset
df = pd.read_csv('udel-churn-train.csv')

In [5]:
# choose the features to use: 'gender', 'SeniorCitizen', 'tenure', 'PaymentMethod', 'MonthlyCharges', 'Churn'
df_choose = df[['gender','SeniorCitizen','tenure','PaymentMethod','MonthlyCharges','Churn']]

In [7]:
# show the missing values ratio for all features
# you should show for example, the feature 'tenure' has 1.6% missing value
df_choose.isnull().sum()/len(df)

,0
gender,0.007987
SeniorCitizen,0.010827
tenure,0.015974
PaymentMethod,0.016507
MonthlyCharges,0.015619
Churn,0.000000


In [12]:
# given missing values are less than 2%, let's drop all rows with missing values from the train dataset

columns_with_missing_values = ['gender', 'SeniorCitizen', 'tenure', 'PaymentMethod', 'MonthlyCharges']
df_choose.dropna(subset=columns_with_missing_values, inplace=True)


<ipython-input-12-5a72f11561bf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_choose.dropna(subset=columns_with_missing_values, inplace=True)


In [13]:
# double check to make sure there is no missing values
df_choose.isnull().sum()

,0
gender,0
SeniorCitizen,0
tenure,0
PaymentMethod,0
MonthlyCharges,0
Churn,0


In [19]:
# check the churn ratio based on whether the person is a senior citizen or not
# based on the result, come up with a rule-based prediction baseline
# refer to the titanic gender based rule-based baseline
df_choose.groupby('SeniorCitizen')['Churn'].value_counts(normalize=True)


SeniorCitizen  Churn
0.0            No       0.761726
               Yes      0.238274
1.0            No       0.591790
               Yes      0.408210
Name: proportion, dtype: float64

**Assuming senior citizens have a higher churn rate, predict that they all choose churn.**  

- 设置baseline：senior citizen all churn yes. 年轻人all churn no.


In [18]:
# read the test dataset
df_test= pd.read_csv('udel-churn-test.csv')

In [20]:
# create a the new column named 'churn_rule_baseline' and assignment prediction results

df_test['churn_rule_baseline'] = 'No'  # Default prediction for non-senior citizens
df_test.loc[df_test['SeniorCitizen'] == 1, 'churn_rule_baseline'] = 'Yes'

print(df_test[['SeniorCitizen', 'churn_rule_baseline']])


      SeniorCitizen churn_rule_baseline
0                 0                  No
1                 0                  No
2                 0                  No
3                 1                 Yes
4                 0                  No
...             ...                 ...
1404              0                  No
1405              0                  No
1406              0                  No
1407              0                  No
1408              1                 Yes

[1409 rows x 2 columns]


In [23]:
# select only two columns for submission - refer to the sample submission file
df_submit_1 = df_test[['customerID','churn_rule_baseline']]
df_submit_1

,customerID,churn_rule_baseline
0,2773-OVBPK,No
1,8292-TYSPY,No
2,6425-JWTDV,No
3,8623-TMRBY,Yes
4,5955-EPOAZ,No
...,...,...
1404,7746-QYVCO,No
1405,6980-IMXXE,No
1406,8390-FESFV,No
1407,0618-XWMSS,No


In [26]:
# save the dataframe to a csv file for submission
# header specifies the column names in the csv file
# index=False we don't need index to be in the csv file
df_submission_1 = pd.DataFrame({
    'customerID': df_test['customerID'],
    'Churn': df_test['churn_rule_baseline']
})
df_submission_1.to_csv('submit_1.csv', index=False)

Submit the baseline prediction to Kaggle and make a screenshot as the following one (upload the screenshot to Canvas for grading).

Note that my baseline accuracy is 0.715:

<img width="979" src="https://user-images.githubusercontent.com/595772/162588565-4bdf2a69-907a-4310-9f7e-e140925d4457.png">

In [30]:
# start building a decision tree model, which hopefully beats the baseline accuracy of 0.715
# first step is to do one-hot encoding for all categorical features

In [29]:
gender_dummies = pd.get_dummies(df_choose.gender).astype(int)
PaymentMethod_dummies = pd.get_dummies(df_choose.PaymentMethod).astype(int)


df_num = df_choose[['SeniorCitizen', 'tenure', 'MonthlyCharges', 'Churn']]  # choose numerical features
df_train_processed = pd.concat([df_num, gender_dummies, PaymentMethod_dummies], axis=1)  # concatenate all dataframes into one
df_train_processed.head()

,SeniorCitizen,tenure,MonthlyCharges,Churn,Female,Male,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
0,0.0,53.0,61.10,No,0,1,0,0,1,0
1,0.0,4.0,84.80,Yes,0,1,1,0,0,0
2,0.0,8.0,29.35,No,0,1,0,0,0,1
3,0.0,56.0,44.85,No,1,0,0,1,0,0
4,0.0,29.0,26.10,No,1,0,0,0,0,1


After one-hot encoding, you processed df_train_processed should look like the following:
<img width="1011" src="https://user-images.githubusercontent.com/595772/162589134-7e5a5a9e-2155-4bdf-b4c0-9b7ed2d7d597.png">

In [31]:
# get X and y from the processed training dataframe
X = df_train_processed.drop('Churn', axis=1)
y = df_train_processed['Churn']

In [32]:
# train a DT model by using all default settings
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X, y)

DecisionTreeClassifier()

In [39]:
# process test dataset for prediction
# select the same features as in training data: 'customerID', 'gender', 'SeniorCitizen', 'tenure', 'PaymentMethod', 'MonthlyCharges'

df_test2 = pd.read_csv('udel-churn-test.csv')

df_test2 = df_test2[['customerID', 'gender', 'SeniorCitizen', 'tenure', 'PaymentMethod', 'MonthlyCharges']]


In [40]:
# check for missing values
# impute any missing values if any

df_test2.isnull().sum()

,0
customerID,0
gender,0
SeniorCitizen,0
tenure,0
PaymentMethod,0
MonthlyCharges,0


In [45]:
# do one-hot encoding for all categorical features for the testing dataset
gender_dummies = pd.get_dummies(df_test2.gender).astype(int)
PaymentMethod_dummies = pd.get_dummies(df_test2.PaymentMethod).astype(int)

df_num2 = df_test2[['SeniorCitizen', 'tenure', 'MonthlyCharges']]  # choose numerical features
X_test = pd.concat([df_num2, gender_dummies, PaymentMethod_dummies], axis=1)  # concatenate all dataframes into one
X_test.head()

,SeniorCitizen,tenure,MonthlyCharges,Female,Male,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
0,0,67,111.30,0,1,1,0,0,0
1,0,19,55.00,0,1,0,1,0,0
2,0,48,19.25,0,1,0,0,0,1
3,1,51,84.20,0,1,0,0,1,0
4,0,6,20.95,1,0,0,0,0,1


After one-hot encoding, you processed X_test should look like the following:
<img width="968" src="https://user-images.githubusercontent.com/595772/162589387-810c34b6-96fc-42dd-8e26-a693d3bae424.png">

In [46]:
# make prediction using the default decision tree and save the results in y_hat
y_hat = tree_clf.predict(X_test)
y_hat

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [48]:
# combine the final dataframe from test dateset and y_hat
# some hints are given below

df_submit_dt = pd.DataFrame({
                              'customerID': df_test2.customerID,
                              'Churn': y_hat,
                            })
df_submit_dt.head()


,customerID,Churn
0,2773-OVBPK,No
1,8292-TYSPY,No
2,6425-JWTDV,No
3,8623-TMRBY,Yes
4,5955-EPOAZ,No


In [49]:
# save the dataframe to CSV and submit to Kaggle for scoring

df_submit_dt.to_csv('submit_simple_dt.csv', index=False)

Submit the default decision tree prediction to Kaggle and save the screenshot for Canvas submission. As you can see my default decision tree only beats the rule-based prediction (0.715) by a slight margin: 0.727


<img width="989" src="https://user-images.githubusercontent.com/595772/162589574-5d0d38e9-3251-4c4e-b8e5-2b8a1a381b09.png">

In [55]:
# now train another decision tree with different parameters: criterion='entropy', max_depth=3
# you can directly use the same X and y

from sklearn import tree

# a tree using entropy and max-depth 3
tree_clf_simple = DecisionTreeClassifier(criterion='entropy', max_depth=3)
tree_clf_simple.fit(X, y)  # a new model is trained using the same data


DecisionTreeClassifier(criterion='entropy', max_depth=3)

In [56]:
# make new predictions using the second decision tree model
# you can reuse X_test

y_hat_2 = tree_clf_simple.predict(X_test)


In [59]:
# combine the final dataframe from test dateset and y_hat_2
# then save the dataframe to CSV and submit to Kaggle for scoring

df_submit_dt2 = pd.DataFrame({
                              'customerID': df_test2.customerID,
                              'Churn': y_hat_2,
                            })
df_submit_dt2.to_csv('submit_simple_dt2.csv', index=False)

submit the simple decision tree prediction results to Kaggle for scoring and save the screenshot for Canvas submission. As you can see, limit the tree depth significantly improved the performance to 0.79 from 0.726:

<img width="971" src="https://user-images.githubusercontent.com/595772/162590450-9466da2f-9c64-4155-92d1-c2bb529ff645.png">
